# High Speed E-Stop CCW/Rotator Integration Test

This notebook performs a high speed E-Stop interlock scenario integration test between the Camera Cable Wrap (CCW) and the Rotator with the Camera Cable Wrap tracking the Rotator. It includes enough boilerplate to allow the test to run at any time by getting current time information from the pointing and computing appropriate coordinates to slew.

In [ ]:
import logging
import yaml

import numpy as np
from matplotlib import pyplot as plt
import astropy.units as u
from astropy.time import Time
from astropy.coordinates import AltAz, ICRS, EarthLocation, Angle, FK5
import asyncio
from lsst.ts import salobj
from lsst.ts.idl.enums import ATPtg

from astropy.utils import iers
iers.conf.auto_download = False

In [ ]:
test_message = "E-Stop MTPtg_Rotator_CCW Integration Test"

In [ ]:
d = salobj.Domain()

In [ ]:
script = salobj.Controller("Script", index=1)
rot = salobj.Remote(d, "Rotator")
#mtptg = salobj.Remote(d, "MTPtg")

In [ ]:
await asyncio.gather(rot.start_task,
                     script.start_task)

In [ ]:
await asyncio.sleep(1.)
#await salobj.set_summary_state(mtptg, salobj.State.ENABLED)
await salobj.set_summary_state(rot, salobj.State.ENABLED)

# Move to 0 deg Starting Position

In [ ]:
await asyncio.sleep(0.1)

print("Move to 0.0 deg for starting position")

await rot.cmd_positionSet.set_start(angle=0.0,timeout=10.)

rot.evt_inPosition.flush()

await rot.cmd_move.start(timeout=30.)

inPOS = await rot.evt_inPosition.aget()

print(inPOS.inPosition)

print("Rotator is at 0 deg starting position")

# Send Track Command to +10 deg

In [ ]:
await asyncio.sleep(1.)

print("Track to +10 deg position")

tel = rot.tel_Application.get()
cpos = tel.Position

vel = 0.068
dt = (10 - cpos) / vel
dpos = vel * 0.1
steps = int(dt / 0.1)

try:
    await rot.cmd_trackStart.start(timeout=30.)
except salobj.AckError:
    pass

In [ ]:
for i in range(steps):
    pos = cpos + i*dpos
    await rot.cmd_track.set_start(
        angle=pos,
        velocity=vel,
        tai=salobj.current_tai(),
        timeout=10.
    )
    await asyncio.sleep(0.1)
    
await rot.cmd_stop.start(timeout=30.)